In [1]:
!pip install pyspark findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [16]:
spark = (
    SparkSession.builder.master('local[*]')
    .appName('Curso PySpark part04')
    .config('spark.sql.repl.eagerEval.enabled', True)
    .getOrCreate()
)

In [ ]:
spark

In [8]:
autores = spark.read.parquet('./DATASETS/AUTORES.parquet')
livros = spark.read.parquet('./DATASETS/LIVROS.parquet')
clientes = spark.read.parquet('./DATASETS/CLIENTES.parquet')
compras = spark.read.parquet('./DATASETS/COMPRAS.parquet')

In [9]:
autores.show(20, False)

+---------+----------------------------+--------------------------+
|id       |titulo                      |autor                     |
+---------+----------------------------+--------------------------+
|106042959|1984                        |George Orwell             |
|017551704|200 Crônicas Escolhidas     |Rubem Braga               |
|054099410|A Alma Encantadora das Ruas |João do Rio               |
|128422117|A Biblioteca da Meia-Noite  |Matt Haig                 |
|049141399|A Coleira do Cão            |Rubem Fonseca             |
|065490862|A Cor Púrpura               |Alice Walker              |
|029077001|A Educação Sentimental      |Gustave Flaubert          |
|032275599|A Escrava Isaura            |Bernardo Guimarães        |
|090690941|A Estrela Sobe              |Marques Rebelo            |
|095695255|A Hora da Estrela           |Clarice Lispector         |
|036365230|A Hora dos Ruminantes       |José J. Veiga             |
|069146824|A Letra Escarlate           |Nathanie

In [11]:
autores.join(other=livros, on='id', how='left').show(20, False)

+---------+----------------------------+--------------------------+------------------+---------------+-------------+-------------+--------------+------+
|id       |titulo                      |autor                     |cnpj_editora      |data_lancamento|ean          |isbn10       |numero_paginas|preco |
+---------+----------------------------+--------------------------+------------------+---------------+-------------+-------------+--------------+------+
|106042959|1984                        |George Orwell             |null              |null           |null         |null         |null          |null  |
|017551704|200 Crônicas Escolhidas     |Rubem Braga               |14.857.032/0001-12|2018-10-19     |2352660010751|0-7433-6026-5|645           |44.38 |
|054099410|A Alma Encantadora das Ruas |João do Rio               |null              |null           |null         |null         |null          |null  |
|128422117|A Biblioteca da Meia-Noite  |Matt Haig                 |null           

In [12]:
autores.join(other=livros, on='id', how='right').show(20, False)

+---------+---------------------------------+-------------------+------------------+---------------+-------------+-------------+--------------+------+
|id       |titulo                           |autor              |cnpj_editora      |data_lancamento|ean          |isbn10       |numero_paginas|preco |
+---------+---------------------------------+-------------------+------------------+---------------+-------------+-------------+--------------+------+
|102961160|null                             |null               |54.317.982/0001-79|2011-09-19     |4367115749184|0-06-594558-1|516           |113.84|
|043636240|null                             |null               |16.938.042/0001-08|2019-05-11     |1239569754256|0-611-23680-X|371           |207.67|
|052928059|null                             |null               |57.624.038/0001-52|2003-08-03     |5462162528221|1-109-29542-1|198           |107.89|
|088482271|null                             |null               |16.249.378/0001-63|2022-09-07

In [ ]:
compras.join(other=livros, on=compras.cd_livro == livros.id, how='inner')

In [23]:
cond = [compras.cd_livro == livros.id, livros.preco < 20, compras.cartao_bandeira == 'American Express']

In [ ]:
compras.join(livros, cond, 'inner')

In [25]:
df1 = autores.where(autores.id.isin('106042959', '017551704', '054099410'))

In [27]:
df2 = livros.where(livros.id.isin('017551704'))

In [28]:
# Inner (O que tem de igual no centro)
df1.join(df2, 'id', 'inner')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
017551704,200 Crônicas Esco...,Rubem Braga,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38


In [29]:
# Left Join (Tudo que ta no 1 data frame, mas o centro dos dois dataframe)
df1.join(df2, 'id', 'left')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
106042959,1984,George Orwell,null,null,null,null,null,null
017551704,200 Crônicas Esco...,Rubem Braga,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38
054099410,A Alma Encantador...,João do Rio,null,null,null,null,null,null


In [30]:
# Right Join (Tudo que ta no 2 data frame, mas o centro dos dois dataframe)
df1.join(df2, 'id', 'right')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
017551704,200 Crônicas Esco...,Rubem Braga,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38


In [31]:
df3 = livros.where(livros.id.isin('017551704', '028125446', '049806867'))
df3

id,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
028125446,90.425.178/0001-77,2011-07-25,2226757040245,0-657-66391-3,384,156.49
049806867,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97
017551704,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38


In [32]:
# Full Join (Vai juntar os dois dataframes)
df1.join(df3, 'id', 'full')

id,titulo,autor,cnpj_editora,data_lancamento,ean,isbn10,numero_paginas,preco
017551704,200 Crônicas Esco...,Rubem Braga,14.857.032/0001-12,2018-10-19,2352660010751,0-7433-6026-5,645,44.38
028125446,null,null,90.425.178/0001-77,2011-07-25,2226757040245,0-657-66391-3,384,156.49
049806867,null,null,93.617.240/0001-10,2022-02-17,3166123637398,1-4822-8784-6,378,162.97
054099410,A Alma Encantador...,João do Rio,null,null,null,null,null,null
106042959,1984,George Orwell,null,null,null,null,null,null


In [ ]:
# Cruza os registros
df1.join(df3, how='cross').orderBy('titulo')

In [35]:
# Anti join pega os registros exclusivos do dataframe A
df1.join(df3, 'id', 'anti')

id,titulo,autor
106042959,1984,George Orwell
054099410,A Alma Encantador...,João do Rio


In [ ]:
df4 = df3.union(df3)
df4

In [37]:
# Tem a mesma função do Inner Join só que remove duplicidade
df1.join(df4, 'id', 'semi')

id,titulo,autor
017551704,200 Crônicas Esco...,Rubem Braga
